# Large Donuts - SITCOM-476

This notebook contains the execution of the Large Donut SITCOM-476 test described in https://jira.lsstcorp.org/browse/SITCOM-476, expected to run during the 2022-10A AuxTel Observing Run. 
                
This notebook is organized in  sections:

    1.1 Setup
    1.2 Conditions assessment
    1.3. Choose target from list of targets in ticket
    1.4. CWFS nearby
    1.5. Slew to target object.
    1.6  Perform the data acquisition sequence. 

-----
## Setup

### Import libraries

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

### Setting up logger

In [ ]:
logger = logging.getLogger("SITCOM-476")
log.level = logging.DEBUG

### Getting unique index for script

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = int(date + str(np.random.randint(0, 9)))

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [ ]:
script.log.info(f'START- SITCOM-476 Large Donuts -- at {Time.now()}')

----
## Assess that conditions meet criteria
Seeing? 
Photometry?
None?

----
## Choose target: Source needs to be above 70 degrees to safely move the hexapod z-axis the requested amount to obtain large donuts. See plots attached by Eske to choose the target, depending on the time of the night. 

### Declare target and filter

In [ ]:
target = 

In [ ]:
filter_to_use = 

------
## CWFS
A CWFS is recommended just before the test to ensure the system is in focus. Based on the target, choose a nearby target to perform CWFS on. 

### Slew to target

#### Declare target 

Query for a target with a magnitud limit of Vmag of 8 and around the same area of the sky as the target source.

In [ ]:
cwfs_target = await script.atcs.find_target(az=180.2, el=75, mag_limit=8)
logger.info(f'Target for CWFS with mag_lim 8 is {cwfs_target}')

#### Slew to the cwfs target

In [ ]:
await script.atcs.slew_object(cwfs_target, rot_type=RotType.PhysicalSky)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
# exposure = await latiss.take_acq(
#         exptime=5, n=1, filter=filter_to_use, grating='empty_0', reason='Acquisition', program="SITCOM-476")
# logger.info(f'Acquisition exposure is {exposure}')

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": filter_to_use, 
                                "grating": 'empty_0',
                                "exposure_time": 20,})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

### Stop tracking:If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
# await script.atcs.stop_tracking()

-------
## Slew to target object

### Confirm that the target is correct

In [1]:
logger.info(f'Target is {target}')

NameError: name 'logger' is not defined

### Slew to the target object

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

To offset the telescope and move the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

----
## Data Acquisition Sequence

### Declare z offset 

In [ ]:
z_offset = 7.5

### Check focus summary

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(f'Original focus offset is \n {original_focus_offset}')

### Going extra focus with +z_offset

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset)

###  Check signal level : Take an image and make sure few thousand counts per pixel in the illuminated annulus are present. 

In [ ]:
saturation_test = await latiss.take_engtest(
        exptime=30, n=1, filter=filter_to_use, grating='empty_0', reason='Exposure_Time_Test', program = "SITCOM-476")
logger.info(f'Saturation test exposure is {saturation_test}')

### Data Acquisition Extra Focus

In [ ]:
extra_images = await latiss.take_focus(
    exptime=20, n=5, filter=filter_to_use, grating='empty_0', reason='Large_Donuts_extra', program ="SITCOM-476")

### Going intra focus with -z_offset

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=-2*z_offset)

### Data Acquisition Intra Focus

In [ ]:
extra_images = await latiss.take_focus(
    exptime=20, n=5, filter=filter_to_use, grating='empty_0', reason='Large_Donuts_intra', program ="SITCOM-476")

### Clear z-offset

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset)

### Confirm the focus offset is back to where it was and wrap up. 

In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(current_focus_offset)

In [ ]:
# Original and current focus offset's user comparison 

In [ ]:
logger.info(f'Current {current_focus_offset.userApplied} focus vs. original {original_focus_offset.userApplied} focus offsets')

In [ ]:
script.log.info(f'END- SITCOM-476 Large Donuts -- at {Time.now()}')